# 第3章: 役割の割り当て（ロールプロンプティング）

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## レッスン

Claudeがあなたの発言以外にコンテキストを持たないというテーマを続けると、時には**Claudeに特定の役割を担ってもらう（必要なコンテキストをすべて含めて）ことが重要**になります。これはロールプロンプティングとしても知られています。役割のコンテキストが詳細であればあるほど良いでしょう。

**Claudeに役割を与えることで、様々な分野でClaudeのパフォーマンスを向上させることができます**。文章作成からコーディング、要約まで幅広く効果があります。これは人間が「______のように考えて」と言われることで助けられることがあるのと似ています。ロールプロンプティングは、Claudeの応答のスタイル、トーン、方法も変更できます。

**注意：** ロールプロンプティングは、システムプロンプトまたはUserメッセージターンの一部として実行できます。

### 例

以下の例では、ロールプロンプティングなしでは、スケートボードについて一文で見解を求められた時、Claudeは**率直で特別な文体でない回答**を提供することがわかります。

しかし、Claudeに猫の役割を担わせると、Claudeの視点が変わり、そのため**Claudeの応答のトーン、スタイル、内容が新しい役割に適応します**。

**注意：** 使える追加のテクニックとして、**Claudeに想定される読者についてのコンテキストを提供する**ことができます。以下では、プロンプトを調整してClaudeが誰に話しかけるべきかも伝えることができました。「あなたは猫です」は「あなたはスケートボーダーの群衆に話しかける猫です」とは全く異なる応答を生み出します。

以下は、システムプロンプトでロールプロンプティングを使わないプロンプトです：

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

以下は同じユーザーの質問ですが、ロールプロンプティングを使用しています。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

ロールプロンプティングを使って、Claudeに特定のスタイルで文章を書いてもらったり、特定の声で話してもらったり、回答の複雑さを導いたりすることができます。**ロールプロンプティングはClaudeの数学や論理タスクの実行能力も向上させることができます。**

例えば、以下の例では明確に正しい答えがあり、それは「はい」です。しかし、Claudeは間違いを犯し、情報が不足していると考えますが、実際にはそうではありません：

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

では、**Claudeに論理ボットとして動作するよう促した場合**はどうでしょうか？Claudeの答えはどのように変わるでしょうか？

この新しい役割の割り当てにより、Claudeは正解を得ることがわかります。（ただし、必ずしもすべて正しい理由ではありませんが）

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**注意：** このコースを通して学ぶことは、**似たような結果を得るために使えるプロンプトエンジニアリング技術が多数ある**ということです。どの技術を使うかはあなたとあなたの好み次第です！**実験して自分自身のプロンプトエンジニアリングスタイルを見つけることをお勧めします**。

上記の内容を変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの最下部にある[**サンプル実験場**](#example-playground)にスクロールしてください。

---

## 演習
- [演習 3.1 - 数学の訂正](#exercise-31---math-correction)

### 演習 3.1 - 数学の訂正
場合によっては、**Claudeは数学、それも簡単な数学で苦労することがあります**。以下では、Claudeは数学問題を正しく解けていると誤って評価していますが、実際には2番目のステップに明らかな計算間違いがあります。Claudeは実際にステップバイステップで進む際に間違いに気づいていますが、全体的な解答が間違っているという結論には至りません。

`PROMPT`および/または`SYSTEM_PROMPT`を修正して、Claudeが解答を正しく解けていると評価するのではなく、`incorrectly`（間違って）解けていると評価するようにしてください。

In [ ]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_3_1_hint)

### おめでとうございます！

この時点ですべての演習を解けていれば、次の章に進む準備ができています。プロンプト作成を楽しんでください！

---

## サンプル実験場

これは、このレッスンで示されたプロンプトサンプルを自由に試し、プロンプトを調整してClaudeの応答にどのような影響があるかを確認するためのエリアです。

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))